In [2]:
import pandas as pd
import plotnine as p9
import numpy as np
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'sans-serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
df = pd.read_csv("microbenchmark.csv")
base_theme = (p9.theme_classic(base_size=14, base_family='sans-serif')
              + p9.theme(axis_text_y=p9.element_text(margin={'r': 5}),
                         legend_box_spacing=0.01,
                         legend_box_margin=0,
                         legend_margin=0,
                         legend_position='top',
                         figure_size=(10,5),
                         text=p9.element_text(size=14)))

In [2]:
def scheme(mode, *colors):
    colordef = {'lred': '#fab7b7', 
                'dred': '#d45079',
                'dblue': '#6886c5', 
                'lblue': '#b0deff', 
                'dorange': '#f5b971',
                'lorange': '#f6d186',
                'dpurp': '#a6b1e1',
                'yellow': '#fcf876',
                'lpurp': '#dcd6f7',
                'lgreen': '#a8e6cf',
                'blackish': '#39375b',
                'seafoam': '#a0ffe6'}
    scfun = p9.scale_fill_manual if mode == 'fill' else p9.scale_color_manual
    if bool(colors):
        return scfun([(colordef[x] if x in colordef else x) for x in colors])
    else:
        return scfun(list(colordef.values()))

In [3]:
def cycles_for(df, op):
    return (df[df.operation == op]
            .groupby(['fs', 'size'])
            .median().apply(np.ceil)
            .cycles.reset_index(name="cycles"))

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [55]:
(p9.ggplot(df.pipe(cycles_for, 'read')) 
 + p9.aes(x='size', y='cycles', color='fs') 
 + p9.geom_line(size=1, alpha=0.7)
 + p9.geom_point()
 + base_theme
 + p9.scale_x_continuous(breaks=[1024, 10240, 102400])
 + scheme('color', 'dblue', 'dred')
 + p9.theme(figure_size=(10,5))
 + p9.labs(x='Size (Bytes)', y='CPU cycles', color='Implementation')).save("plots/read.png", dpi=300)

In [56]:
(p9.ggplot(df.pipe(cycles_for, 'write')) 
 + p9.aes(x='size', y='cycles', color='fs') 
 + p9.geom_line(size=1, alpha=0.7)
 + p9.geom_point()
 + base_theme
 + p9.scale_x_continuous(breaks=[1024, 10240, 102400])
 + scheme('color', 'dblue', 'dred')
 + p9.theme(figure_size=(10,5))
 + p9.labs(x='Size (Bytes)', y='CPU cycles', color='Implementation')).save("plots/write.png", dpi=300)

In [16]:
medians = df.groupby(['fs', 'operation', 'size']).median().apply(np.ceil).cycles.reset_index(name='cycles')
devs = df.groupby(['fs', 'operation', 'size']).std().apply(lambda x: round(x,2)).cycles.reset_index(name='stdev')
summary = pd.DataFrame({'fs': medians.fs,
                        'op': medians.apply(lambda x: x.operation if x['size'] < 0 else ' '.join([x.operation, str(x['size'])]), axis='columns'),
                        'cycles': medians.cycles.map(int),
                        'stdev': devs.stdev.map(int)}).pivot(index='fs', columns='op', values=['cycles', 'stdev'])
summary.to_latex("test.tex")

In [48]:
for idx, row in summary.stack().iterrows():
    print(re.sub(' ', '-', '-'.join(idx)), row.cycles, row.stdev)

adafs-create 548615020 16668215
adafs-read-1 5852 733
adafs-read-1024 2165 154
adafs-read-10240 2172 3813
adafs-read-102400 2129 166
adafs-remove 516977 146179
adafs-write-1 2095 226
adafs-write-1024 1953 190
adafs-write-10240 1917 372
adafs-write-102400 2014 326
fuse-mfs-create 1329948 1709262
fuse-mfs-read-1 5144 3262
fuse-mfs-read-1024 1713 525
fuse-mfs-read-10240 1698 537
fuse-mfs-read-102400 1703 516
fuse-mfs-remove 208077 267264
fuse-mfs-write-1 1898 492
fuse-mfs-write-1024 1621 403
fuse-mfs-write-10240 1744 358
fuse-mfs-write-102400 1779 348


In [49]:
import re
for idx, row in summary.stack().iterrows():
    fname = f"stats/{re.sub(' ', '-', '-'.join(idx))}.tex"
        
    with open(fname, "w") as f:
        f.write(str(int(row.cycles)))
    
    fname = f"stats/{re.sub(' ', '-', '-'.join(idx))}-stdev.tex"
    with open(fname, "w") as f:
        f.write(str(int(row.stdev)))


In [50]:
summary.loc['difference'] = summary.apply(lambda x: x.adafs - x['fuse-mfs'])
summary.loc['c:ada'] = summary.apply(lambda x: f"1:{round(x.adafs/x['fuse-mfs'],2)}")

In [51]:
summary

cycles                                                   \
op             create  read 1 read 1024 read 10240 read 102400  remove   
fs                                                                       
adafs       548615020    5852      2165       2172        2129  516977   
fuse-mfs      1329948    5144      1713       1698        1703  208077   
difference  547285072     708       452        474         426  308900   
c:ada        1:412.51  1:1.14    1:1.26     1:1.28      1:1.25  1:2.48   

                                                           stdev          \
op         write 1 write 1024 write 10240 write 102400    create  read 1   
fs                                                                         
adafs         2095       1953        1917         2014  16668215     733   
fuse-mfs      1898       1621        1744         1779   1709262    3262   
difference     197        332         173          235  14958953   -2529   
c:ada        1:1.1      1:1.2       1:1.1       1:1.13    1:9.75  1:0.22   

                                                                         \
op         read 1024 read 10240 read 102400   remove write 1 write 1024   
fs                                                                        
adafs            154       3813         166   146179     226        190   
fuse-mfs         525        537         516   267264     492        403   
difference      -371       3276        -350  -121085    -266       -213   
c:ada         1:0.29      1:7.1      1:0.32   1:0.55  1:0.46     1:0.47   

                                     
op         write 10240 write 102400  
fs                                   
adafs              372          326  
fuse-mfs           358          348  
difference          14          -22  
c:ada           1:1.04       1:0.94

In [22]:
io_df = pd.read_csv("io_comparisons.csv").applymap(int)

In [23]:
differences = pd.DataFrame({
    'direct-io': {'cycles': io_df.direct_io.median(), 'stdev': io_df.direct_io.std()},
    'stream-io': {'cycles': io_df.stream_io.median(), 'stdev': io_df.stream_io.std()},
}).astype({'direct-io': np.int64, 'stream-io': np.int64}).transpose()
differences

,cycles,stdev
direct-io,1953586,190209
stream-io,1463269,664984


In [24]:
import re
for idx, row in differences.iterrows():
    fname = f"stats/{idx}.tex"
        
    with open(fname, "w") as f:
        f.write(str(int(row.cycles)))
    
    fname = f"stats/{idx}-stdev.tex"
    with open(fname, "w") as f:
        f.write(str(int(row.stdev)))

In [38]:
differences.at['cycles', 'direct']-differences.at['cycles', 'stream']

490317